In [1]:
# # verify GPU availability
# import tensorflow as tf

# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [2]:
! nvidia-smi

zsh:1: command not found: nvidia-smi


In [4]:
!pip install pytorch-lightning

     |████████████████████████████████| 922 kB 1.5 MB/s eta 0:00:01


In [7]:

configuration

BartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 1024,
  "model_type": "bart",
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "scale_embedding": false,
  "transformers_version": "4.10.0",
  "use_cache": true,
  "vocab_size": 50265
}

In [1]:
# BERT imports
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm, trange
import pandas as pd
import io
import os
import numpy as np
# import matplotlib.pyplot as plt
# % matplotlib inline

# specify GPU device
import pytorch_lightning as pl

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# n_gpu = torch.cuda.device_count()
# torch.cuda.get_device_name(0)

In [2]:
pl.seed_everything(42)

Global seed set to 42


42

In [3]:
file_path = 'data/nikhil_onlytextpytorch.json'

path = os.path.join(os.getcwd(),file_path)

train = pd.read_json(path, orient='index').reset_index()
display(train)

,index,answer,question,context,source
0,"What values are specified to replaceNaN, posit...","bynan,posinf, andneginf","What values are specified to replaceNaN, posit...","ReplacesNaN, positive infinity, and negative i...",https://pytorch.org/docs/stable/generated/torc...
1,"By default,NaN is replaced with what value?",zero,"By default,NaN is replaced with what value?","ReplacesNaN, positive infinity, and negative i...",https://pytorch.org/docs/stable/generated/torc...
2,What values are used to replace negative infin...,"bynan,posinf, andneginf",What values are used to replace negative infin...,"ReplacesNaN, positive infinity, and negative i...",https://pytorch.org/docs/stable/generated/torc...
3,What is the default value for negative infinity?,the least finite value,What is the default value for negative infinity?,"ReplacesNaN, positive infinity, and negative i...",https://pytorch.org/docs/stable/generated/torc...
4,What is the value to replace positive infinity...,posinf,What is the value to replace positive infinity...,"nan(Number,optional) – the value to replaceNaN...",https://pytorch.org/docs/stable/generated/torc...
...,...,...,...,...,...
998,What is the tensor of shape where*is zero or m...,A(Tensor),What is the tensor of shape where*is zero or m...,torch.linalg.inv()computes the inverse of a sq...,https://pytorch.org/docs/stable/generated/torc...
999,What is the tolerance value to determine when ...,rcond,What is the tolerance value to determine when ...,torch.linalg.lstsq()computesA.pinv() @Bwith a\...,https://pytorch.org/docs/stable/generated/torc...
1000,What is the default value of atorch.Tensor?,1e-15,What is the default value of atorch.Tensor?,See also torch.linalg.inv()computes the invers...,https://pytorch.org/docs/stable/generated/torc...
1001,What computes the inverse of a square matrix?,torch.linalg.inv(),What computes the inverse of a square matrix?,See also torch.linalg.inv()computes the invers...,https://pytorch.org/docs/stable/generated/torc...


In [4]:
from transformers import BertTokenizer, BertModel


class GetEncodings:
    def __init__(self):
        pass

    def encode(text, tokenizer, model, max_length=512):
        '''Tokenize data and get encoded embeddings from model'''
        tk = tokenizer(
                text,
                max_length=max_length,
                padding='max_length',
                return_attention_mask=True,
                add_special_tokens=True,
                truncation=True,
                return_tensors="pt"
            )
        model_op = model(**tk)
        return model_op

In [10]:
import faiss

class SearchSimilar:
    def __init__(self, embeddings, shape):

        self.embeddings = embeddings.cpu().detach().numpy()
        self.index = faiss.index_factory(shape, "Flat", faiss.METRIC_INNER_PRODUCT)
        self.total = self.index.ntotal
        
        faiss.normalize_L2(self.embeddings)
        self.index.add(self.embeddings)

    def get_n_similar_vectors(self, text, n=2):
        '''Find the top n similar vector from text using cosine similarity FAISS'''
        text = text.cpu().detach().numpy()
        faiss.normalize_L2(text)
        distance, index = self.index.search(text, n)
        return distance, index
    
    def get_n_dissimilar_vectors(self, text, n=2):
        '''Find the top n dissimilar vector from text using cosine similarity FAISS'''
        pass

In [5]:
dict1 = torch.load('data/context_encodings/file_100.pt')
dict2 = torch.load('data/context_encodings/file_200.pt')
dict3 = torch.load('data/context_encodings/file_300.pt')
dict4 = torch.load('data/context_encodings/file_400.pt')

dict1.update(dict2)
dict1.update(dict3)
dict1.update(dict4)



In [6]:
import gc
gc.collect()

18

In [ ]:


import torch

class TextTokenizer(torch.utils.data.Dataset):
    def __init__(
        self,
        data:pd.DataFrame, 
        tokenizer:BertTokenizer,
        column_name:str,
        source_max_length: int=24,
        context_max_length: int=320,
        answer_max_length: int=30
        
        ):
        
        
        self.tokenizer = tokenizer
        self.data = data
        self.question_max_length = question_max_length
        self.answer_max_length = answer_max_length
        self.context_max_length = context_max_length

        self.column_name = column_name


    def __len__(self):
        return len(self.data)


    def __getitem__(self, index:int):
        data_row = self.data.iloc[index]
        question_encoding = tokenizer(
            data_row['question'],
            max_length=self.source_max_length,
            padding='max_length',
            return_attention_mask=True,
            add_special_tokens=True,
            truncation=True,
            return_tensors="pt")
        
        context_encoding = tokenizer(
            data_row['answer'],
            max_length=self.source_max_length,
            padding='max_length',
            return_attention_mask=True,
            add_special_tokens=True,
            truncation=True,
            return_tensors="pt")
        
        similar_context = get_closest_context(data_row['answer'], n=2)
        
        
        answer_encoding = tokenizer(
            data_row['answer'],
            max_length=self.source_max_length,
            padding='max_length',
            return_attention_mask=True,
            add_special_tokens=True,
            truncation=True,
            return_tensors="pt")
        
        


        return dict(
            input_ids_questions_merged = question_encoding['input_ids'].flatten(),
            attention_mask_questions_merged = question_encoding['attention_mask'].flatten(),
            input_ids_answers = answer_encoding['input_ids'].flatten(),
            attention_mask_answers = answer_encoding['attention_mask'].flatten()
        )






In [6]:
value_ver = dict1[1]['model_op']['pooler_output']

In [7]:
final_op = [i['model_op']['pooler_output'] for i in dict1.values()]

In [8]:
final_op = torch.cat(final_op, 0)

In [11]:
final_op.shape

torch.Size([347, 768])

In [ ]:
ss = SearchSimilar(final_op, 768)

In [17]:
d, i = ss.get_n_similar_vectors(value_ver, 3)

In [16]:
# install huggingface BART MOdel
from transformers import BartModel, BartConfig

# Initializing a BART facebook/bart-large style configuration
configuration = BartConfig()

# Initializing a model from the facebook/bart-large style configuration
model = BartModel(configuration)

# Accessing the model configuration
configuration = model.config

In [30]:
i[0].tolist()

[9, 10, 0]

In [8]:

from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs


In [3]:
import torch
data_sample = torch.load('file_100.pt')


In [ ]:

class BART_Model(torch.nn.Module):
    def __init__(self):
        super(TrainBART, self).__init__()
        configuration = BartConfig()
        self.bart_model = BartModel(configuration)
    
    def forward(self, source, target):

        bart_outputs = self.bart_model(source, target)        
        
        return question_outputs
    
    
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import Trainer, TrainingArguments
from transformers.modeling_bart import shift_tokens_right

dataset = ... # some Datasets object with train/validation split and columns 'text' and 'summary'
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['text'], pad_to_max_length=True, max_length=1024, truncation=True))
    target_encodings = tokenizer.batch_encode_plus(example_batch['summary'], pad_to_max_length=True, max_length=1024, truncation=True))
    
    labels = target_encodings['input_ids']
    decoder_input_ids = shift_tokens_right(labels, model.config.pad_token_id)
    labels[labels[:, :] == model.config.pad_token_id] = -100
    
    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'decoder_input_ids': decoder_input_ids,
        'labels': labels,
    }

    return encodings

dataset = dataset.map(convert_to_features, batched=True)
columns = ['input_ids', 'labels', 'decoder_input_ids','attention_mask',] 
dataset.set_format(type='torch', columns=columns)

training_args = TrainingArguments(
    output_dir='./models/bart-summarizer',          
    num_train_epochs=1,           
    per_device_train_batch_size=1, 
    per_device_eval_batch_size=1,   
    warmup_steps=500,               
    weight_decay=0.01,              
    logging_dir='./logs',          
)

trainer = Trainer(
    model=model,                       
    args=training_args,                  
    train_dataset=dataset['train'],        
    

In [ ]:
model = EnsembleTokens().to(device)

In [ ]:

question_encoding, answer_encoding = model(next(iter(train_loader)))

In [ ]:
list(model.children())[0]

In [ ]:

from torch.optim import AdamW

criterion = torch.nn.CosineSimilarity()

learning_rate = 5e-5
adam_epsilon=1e-8
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)

In [ ]:


num_train_epochs = 10
fraction_per_epoch = 0.2

print("***** Running training *****")
print("  Num examples = %d" % len(dataset))
print("  Num Epochs = %d" % num_train_epochs)
print("  Batch size = %d" % BATCH_SIZE)
print("  Total optimization steps = %d" % (len(train_loader) // (num_train_epochs * fraction_per_epoch)))

model.zero_grad()
train_iterator = trange(num_train_epochs, desc="Epoch")

train_loss_set = []
loss = 0
for _ in train_iterator:
    print(loss)
    epoch_iterator = tqdm(iter(train_loader), desc="Iteration")
    for step, batch in enumerate(epoch_iterator):
    #   if step < global_step + 1:
    #     continue

      model.train()
    #   batch = tuple(t.to(device) for t in batch)

      question_encoding, answer_encoding = model(batch)
      loss = 1. - criterion(question_encoding[0], answer_encoding[0])
      train_loss_set.append(loss)
      print(f'####$$$$$$$%%%%%% Loss  {loss.mean()}')
    #   print(loss, loss.shape)
      loss = loss.mean().backward()
      
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    #   tr_loss += loss.item()
      optimizer.step()
      model.zero_grad()
    #   global_step += 1
    
    #   if global_step % 1000 == 0:
    #     print("Train loss: {}".format(tr_loss/global_step))
    #     output_dir = 'checkpoint-{}'.format(global_step)
    #     if not os.path.exists(output_dir):
    #         os.makedirs(output_dir)
    #     model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    #     model_to_save.save_pretrained(output_dir)
    #     torch.save(torch.tensor(train_loss_set), os.path.join(output_dir, 'training_loss.pt'))
    #     print("Saving model checkpoint to %s" % output_dir)
